In [ ]:
class Variable():

    ACROSS = "across"
    DOWN = "down"

    def __init__(self, i, j, direction, length):
        """Create a new variable with starting point, direction, and length."""
        self.i = i
        self.j = j
        self.direction = direction
        self.length = length
        self.cells = []
        for k in range(self.length):
            self.cells.append(
                (self.i + (k if self.direction == Variable.DOWN else 0),
                 self.j + (k if self.direction == Variable.ACROSS else 0))
            )

    def __hash__(self):
        return hash((self.i, self.j, self.direction, self.length))

    def __eq__(self, other):
        return (
            (self.i == other.i) and
            (self.j == other.j) and
            (self.direction == other.direction) and
            (self.length == other.length)
        )

    def __str__(self):
        return f"({self.i}, {self.j}) {self.direction} : {self.length}"

    def __repr__(self):
        direction = repr(self.direction)
        return f"Variable({self.i}, {self.j}, {direction}, {self.length})"


class Crossword():

    def __init__(self, structure_file, words_file):

        # Determine structure of crossword
        with open(structure_file) as f:
            contents = f.read().splitlines()
            self.height = len(contents)
            self.width = max(len(line) for line in contents)

            self.structure = []
            for i in range(self.height):
                row = []
                for j in range(self.width):
                    if j >= len(contents[i]):
                        row.append(False)
                    elif contents[i][j] == "_":
                        row.append(True)
                    else:
                        row.append(False)
                self.structure.append(row)

        # Save vocabulary list
        with open(words_file) as f:
            self.words = set(f.read().upper().splitlines())

        # Determine variable set
        self.variables = set()
        for i in range(self.height):
            for j in range(self.width):

                # Vertical words
                starts_word = (
                    self.structure[i][j]
                    and (i == 0 or not self.structure[i - 1][j])
                )
                if starts_word:
                    length = 1
                    for k in range(i + 1, self.height):
                        if self.structure[k][j]:
                            length += 1
                        else:
                            break
                    if length > 1:
                        self.variables.add(Variable(
                            i=i, j=j,
                            direction=Variable.DOWN,
                            length=length
                        ))

                # Horizontal words
                starts_word = (
                    self.structure[i][j]
                    and (j == 0 or not self.structure[i][j - 1])
                )
                if starts_word:
                    length = 1
                    for k in range(j + 1, self.width):
                        if self.structure[i][k]:
                            length += 1
                        else:
                            break
                    if length > 1:
                        self.variables.add(Variable(
                            i=i, j=j,
                            direction=Variable.ACROSS,
                            length=length
                        ))

        # Compute overlaps for each word
        # For any pair of variables v1, v2, their overlap is either:
        #    None, if the two variables do not overlap; or
        #    (i, j), where v1's ith character overlaps v2's jth character
        self.overlaps = dict()
        for v1 in self.variables:
            for v2 in self.variables:
                if v1 == v2:
                    continue
                cells1 = v1.cells
                cells2 = v2.cells
                intersection = set(cells1).intersection(cells2)
                if not intersection:
                    self.overlaps[v1, v2] = None
                else:
                    intersection = intersection.pop()
                    self.overlaps[v1, v2] = (
                        cells1.index(intersection),
                        cells2.index(intersection)
                    )

    def neighbors(self, var):
        """Given a variable, return set of overlapping variables."""
        return set(
            v for v in self.variables
            if v != var and self.overlaps[v, var]
        )


In [ ]:
import sys
import queue

class CrosswordCreator():

    def __init__(self, crossword):
        """
        Create new CSP crossword generate.
        """
        self.crossword = crossword
        self.domains = {
            var: self.crossword.words.copy()
            for var in self.crossword.variables
        }

    def letter_grid(self, assignment):
        """
        Return 2D array representing a given assignment.
        """
        letters = [
            [None for _ in range(self.crossword.width)]
            for _ in range(self.crossword.height)
        ]
        for variable, word in assignment.items():
            direction = variable.direction
            for k in range(len(word)):
                i = variable.i + (k if direction == Variable.DOWN else 0)
                j = variable.j + (k if direction == Variable.ACROSS else 0)
                letters[i][j] = word[k]
        return letters

    def print(self, assignment):
        """
        Print crossword assignment to the terminal.
        """
        letters = self.letter_grid(assignment)
        for i in range(self.crossword.height):
            for j in range(self.crossword.width):
                if self.crossword.structure[i][j]:
                    print(letters[i][j] or " ", end="")
                else:
                    print("█", end="")
            print()

    def save(self, assignment, filename):
        """
        Save crossword assignment to an image file.
        """
        from PIL import Image, ImageDraw, ImageFont
        cell_size = 100
        cell_border = 2
        interior_size = cell_size - 2 * cell_border
        letters = self.letter_grid(assignment)

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.crossword.width * cell_size,
             self.crossword.height * cell_size),
            "black"
        )
        font = ImageFont.truetype("assets/fonts/OpenSans-Regular.ttf", 80)
        draw = ImageDraw.Draw(img)

        for i in range(self.crossword.height):
            for j in range(self.crossword.width):

                rect = [
                    (j * cell_size + cell_border,
                     i * cell_size + cell_border),
                    ((j + 1) * cell_size - cell_border,
                     (i + 1) * cell_size - cell_border)
                ]
                if self.crossword.structure[i][j]:
                    draw.rectangle(rect, fill="white")
                    if letters[i][j]:
                        _, _, w, h = draw.textbbox((0, 0), letters[i][j], font=font)
                        draw.text(
                            (rect[0][0] + ((interior_size - w) / 2),
                             rect[0][1] + ((interior_size - h) / 2) - 10),
                            letters[i][j], fill="black", font=font
                        )

        img.save(filename)

In [ ]:
crossword = Crossword('data/structure0.txt', 'data/words0.txt')

In [ ]:
crossword.variables

In [ ]:
crossword.overlaps

In [ ]:
crossword.structure

In [ ]:
crossword.height, crossword.width

In [ ]:
crossword.words

In [ ]:
result = CrosswordCreator(crossword)

In [ ]:
result.domains

In [ ]:
for var in result.domains:
    viable_options = {option for option in result.domains[var] if len(option) == var.length}
    result.domains[var] = viable_options
    print(result.domains[var])

In [ ]:
[var for var in result.domains]

In [ ]:
result.domains[Variable(4, 1, 'across', 4)]

In [ ]:
[overlap for overlap in crossword.overlaps if (Variable(4, 1, 'across', 4)) in overlap] 

In [ ]:
crossword.overlaps[Variable(4, 1, 'across', 4), Variable(0, 1, 'down', 5)]

In [ ]:
for var in result.domains[Variable(4, 1, 'across', 4)]:
    print(var)

In [ ]:
def revise(self, x, y):
        """
        Make variable `x` arc consistent with variable `y`.
        To do so, remove values from `self.domains[x]` for which there is no
        possible corresponding value for `y` in `self.domains[y]`.

        Return True if a revision was made to the domain of `x`; return
        False if no revision was made.
        """
        revised = False
        # Identify the relevant word positions for any overlaps
        x_pos, y_pos = self.crossword.overlaps[x, y]
        to_remove = set()
        for val in self.domains[x]:
            # Iterate through the y values, if no words with correct intersect, valid = empty
            valid = [option for option in self.domains[y] if option[y_pos] == val[x_pos]]
            # If valid empty, no viable options for (x, y) that correctly overlap for x
            if not valid:
                to_remove.add(val)
                revised = True

        for val in to_remove:
            self.domains[x].remove(val)
            
        return revised

In [ ]:
revise(result, Variable(4, 1, 'across', 4), Variable(0, 1, 'down', 5))

In [ ]:
import queue

test = queue.Queue()

for val in [1, 2, 3, 4, 5]:
    test.put(val)

for i in range(5):
    print(test.get())

In [ ]:
list(result.domains)

In [ ]:
result.crossword.overlaps

In [ ]:
{k: v for k, v in sorted(result.domains.items(), key=lambda item: len(item[1]), reverse=True)}

In [ ]:
def ac3(self, arcs=None):
        """
        Update `self.domains` such that each variable is arc consistent.
        If `arcs` is None, begin with initial list of all arcs in the problem.
        Otherwise, use `arcs` as the initial list of arcs to make consistent.

        Return True if arc consistency is enforced and no domains are empty;
        return False if one or more domains end up empty.
        """
        # Create a queue of all domains and options sorted by size, revise until domains
        # reduced fully to one option each, or the solution fails
        arcs = queue.Queue()
        domains = {val: domain for val, domain in sorted(self.domains.items(), key=lambda item: len(item[1]), reverse=True)}
        
        for k, v in domains:
            arcs.put((k, v))

        while arcs:
            print(arcs.get())

ac3(result)

In [ ]:
from collections import deque

def a(self, arcs=None):
    variables = [val for val in self.domains.keys()]
    print((variables, variables))
    queue = [(x, y) for x, y in (variables, variables) if x != y] if not arcs else arcs
    return queue

In [ ]:
a(result)

In [ ]:
[overlap for overlap in result.crossword.overlaps if overlap[0] == Variable(4, 1, 'across', 4)]

In [ ]:
result.domains.keys()

In [ ]:
result.crossword.neighbors(Variable(4, 1, 'across', 4))

In [ ]:
test = []
bacon = [True, False]

In [ ]:
for row in bacon:
    test.append(1 if row else 0)

test

In [ ]:
for i in range(2, -1, -1):
    print(i)

In [ ]:
mp = x / 2
if mp ** 2 == x:
    return mp

elif mp ** 2 > x:
    mp /= 2

else:
    mp = (mp + prev) / 2

In [ ]:
def singleNumber(nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        seen = {}
        
        for i in range(len(nums)):
            if nums[i] in seen.keys():
                seen[nums[i]] += 1
            else:
                seen[nums[i]] = 1

        return [k for k,v in seen.items() if v != 2][0]

In [ ]:
singleNumber([4,1,2,1,2])

In [ ]:
import nltk

def preprocess(sentence):
    """
    Convert `sentence` to a list of its words.
    Pre-process sentence by converting all characters to lowercase
    and removing any word that does not contain at least one alphabetic
    character.
    """
    words = nltk.tokenize.word_tokenize(sentence.lower())
    return [word for word in words if word.isalpha()]

In [ ]:
preprocess('when I went to the p4rk I was happy')

In [1]:
import sys
import tensorflow as tf

from PIL import Image, ImageDraw, ImageFont
from transformers import AutoTokenizer, TFBertForMaskedLM

# Pre-trained masked language model
MODEL = "bert-base-uncased"

# Number of predictions to generate
K = 3

# Constants for generating attention diagrams
# FONT = ImageFont.truetype("assets/fonts/OpenSans-Regular.ttf", 28)
GRID_SIZE = 40
PIXELS_PER_WORD = 200

text = input("Text: ")

    # Tokenize input
tokenizer = AutoTokenizer.from_pretrained(MODEL)
inputs = tokenizer(text, return_tensors="tf")

2024-10-06 15:30:14.065120: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-06 15:30:14.239759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-06 15:30:14.284596: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-06 15:30:14.293858: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-06 15:30:14.336102: I tensorflow/core/platform/cpu_feature_guar

Text:  hello


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/home/christian/.pyenv/versions/3.10.6/envs/base_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

I0000 00:00:1728199835.291499   73661 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-06 15:30:35.298682: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
# pip install tf-keras

In [14]:
for val in inputs['input_ids'][0]:
    print(val)

tf.Tensor(101, shape=(), dtype=int32)
tf.Tensor(7592, shape=(), dtype=int32)
tf.Tensor(102, shape=(), dtype=int32)


In [16]:
for i in str(24):
    print(i)

2
4


In [17]:
nums = [2,2]

if nums[0] in nums[1:4]:
    print(True)

True
